In [31]:
from nltk.corpus import wordnet as wn
import nltk

if '/Users/haiphan/Documents/GitHub/WordNet/Hải/nltk_data' not in nltk.data.path:
    nltk.data.path.append('/Users/haiphan/Documents/GitHub/WordNet/Hải/nltk_data')


In [32]:
# Hàm để lấy danh sách các quan hệ (relationship) của một synset
def get_relationships(synset, relationship_type):
    if relationship_type == 'hypernym':
        return synset.hypernyms()
    elif relationship_type == 'hyponym':
        return synset.hyponyms()
    elif relationship_type == 'meronym':
        return synset.part_meronyms() + synset.substance_meronyms()
    elif relationship_type == 'holonym':
        return synset.part_holonyms() + synset.substance_holonyms()
    else:
        return []


# Hàm giải mã lemmas
def process_lemmas(lemmas, recursive_level=None):
    result_list = ''
    if recursive_level is not None:
        result_list += f'Lv{recursive_level + 1}. '
    for i, lemma in enumerate(lemmas):
        result_list += lemma.name()
        if (i + 1) < len(lemmas):
            result_list += '/'
    return result_list


# Hàm để xây dựng biểu đồ cây bằng cú pháp Mermaid với giới hạn đệ quy
def build_mermaid_list(synset, relationship_type, parent_id=None, recursive_level=0, max_recursive=1, result_list=[]):
    # Break condition of recursive
    if recursive_level >= max_recursive:
        return  

    # Get synset_id
    synset_id = synset.name()

    # If the synset got its parents
    if parent_id:
        parent_lemmas = f'*{parent_id}*-{process_lemmas(wn.synset(parent_id).lemmas())}'
        synset_lemmas = f'*{synset_id}*-{process_lemmas(synset.lemmas())}'

        # Hyponym and Meronym
        if relationship_type in ['hyponym', 'meronym']:
            result_list.append(f'{parent_lemmas} --> {synset_lemmas}')
        # Hypernym and Holonym
        else:
            result_list.append(f'{synset_lemmas} --> {parent_lemmas}')

    # If the synset is a leaf
    else:
        synset_lemmas = f'*{synset_id}*-{process_lemmas(synset.lemmas())}'
        result_list.append(f'{synset_lemmas}["{synset_lemmas}"]')

    # Recursive running
    relationships = get_relationships(synset, relationship_type)
    for relation_synset in relationships:
        build_mermaid_list(relation_synset, relationship_type, synset_id, recursive_level + 1, max_recursive, result_list)


# Hàm để xây dựng biểu đồ cây bằng cú pháp Mermaid bằng kết quả của hàm build_mermaid_list
def build_mermaid_tree(synsets, relationship_type, max_recursive=1):
    # Khởi tạo mảng lưu dữ liệu tạo ra
    result_list = []

    # Tìm max_recursive thích hợp để Fix lỗi "Hiển thị MERMAID" (Chỉ cho HYPERNYM)
    if relationship_type == 'hypernym':
        min_level = 20
        max_level = 0
        for synset in synsets:
            # Tìm độ sâu của synset có path ngắn nhất đến "entity"
            hyper_path = synset.hypernym_paths()
            for path in hyper_path:
                if len(path) < min_level:
                    min_level = len(path)
                if len(path) > max_level:
                    max_level = len(path)

        # Cập nhật max_recursive 
        # Nếu có 1 nhánh nào đó chạy tới entity (max_recursive > min_level)
        # => Gán giá trị max_recursive bằng max_level (để chạy từ entity xuống)
        if max_recursive >= min_level:
            max_recursive = max_level

    # Gọi hàm để tạo Mermaid code
    for synset in synsets:
        temp_list = []
        build_mermaid_list(synset=synset, relationship_type=relationship_type, max_recursive=max_recursive, result_list=temp_list)
        result_list += temp_list

    # Loại bỏ trùng lặp
    result_list = list(set(result_list))

    # Cập nhật mermaid_code
    mermaid_code = []
    for item in result_list:
        mermaid_code.append(item)
    
    # Trả về mermaid code để xây dựng cây
    return mermaid_code



In [35]:
a = build_mermaid_tree(wn.synsets('people', pos='n'), 'hypernym', 2)
a

['*group.n.01*-group/grouping --> *people.n.01*-people',
 '*family.n.04*-family/family_line/folk/kinfolk/kinsfolk/sept/phratry --> *people.n.03*-people',
 '*group.n.01*-group/grouping --> *citizenry.n.01*-citizenry/people',
 '*group.n.01*-group/grouping --> *multitude.n.03*-multitude/masses/mass/hoi_polloi/people/the_great_unwashed',
 '*people.n.03*-people["*people.n.03*-people"]',
 '*citizenry.n.01*-citizenry/people["*citizenry.n.01*-citizenry/people"]',
 '*people.n.01*-people["*people.n.01*-people"]',
 '*multitude.n.03*-multitude/masses/mass/hoi_polloi/people/the_great_unwashed["*multitude.n.03*-multitude/masses/mass/hoi_polloi/people/the_great_unwashed"]']